Here I remove genes associated with the by me identified strain specific gene expression.  
LOC119965961, LOC119965947 (variants of 60S ribosomal protein L13a)  
LOC119970178, LOC119969773 (variants of eukaryotic translation initiation factor 6)  

There seem to be 3 different strains of shark that are using a combination of the variants for these genes.   
I calculated a linear regression for all other genes compared to these genes to see if any had a similar expression pattern.  
I then filtered any genes with an R2 score larger than 0.05, removing 272 genes from the dataset.  
After this the purported xbp1 hatching gland cluster merged with the neural crest. Upon closer investigation I could not find a single unique marker for this cluster that was not also a marker of either Neural crest or of the strain. Xbp1 and Chac1 for example mark the cells of the LOC119965961 associated strain.  



In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx, State
import dash_ag_grid as dag

import pandas as pd

import os
import sys
import time
import warnings
import gc
import io
import base64
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
adata=sc.read('h5ad_files/catshark/sScyCan1.1/SE8_9_pass_after_filtering.h5ad')

In [ ]:
adata[:,adata.var.index=='LOC119965947'].X.todense()

In [ ]:
plt.hist(adata[:,adata.var.index=='LOC119965947'].X.todense(),bins = 50)

In [ ]:
plt.hist(adata[:,adata.var.index=='LOC119965947'].layers['original_counts'].todense(),bins = 50,range=(0, 100))

In [ ]:
plt.hist(adata[:,adata.var.index=='LOC119965961'].X.todense(),bins = 50)

In [ ]:
plt.hist(adata[:,adata.var.index=='LOC119965961'].layers['original_counts'].todense(),bins = 50,range=(0, 50))
plt.ylim(0,200)

In [ ]:
strain_cutoff = adata[:,adata.var.index=='LOC119965961'].layers['original_counts'].todense().flatten()<10
adata.obs['strain_specific'] = strain_cutoff.tolist()[0]

In [ ]:
sc.pl.umap(adata,color = 'strain_specific')

In [ ]:
from sklearn.linear_model import LinearRegression
def do_reg(gene,variable):
    X = adata[:,gene].X.toarray() #get the gene expression value
    y = adata[:,gene].obs[variable] #get the variable obs value
    reg = LinearRegression().fit(X, y) #fit linear regression for gene expression and variable value
    #return (variable, gene, reg.score(X, y))
    return reg.score(X, y) #return the r2 score of the linear regression

In [ ]:
#now parallelize the function
from multiprocessing import Pool
import itertools

def do_reg_parallel(gene_list,variable,n_threads): #supply the function with a list of genes and a column in adata.obs with your variable you want to fit gene expression to
    with Pool(n_threads) as p: #if you use too many threads and launching them requires more memory than is available, the processes won't launch
        return p.starmap(do_reg, #use starmap to be able to call both vars required for the do_reg function
                         zip(gene_list, #list of genes
                             itertools.repeat(variable) #repeat the column name for each gene
                            ))
        p.close()
        # wait for all tasks to complete
        p.join()

In [ ]:
adata.obs['strain_LOC119965961'] = adata[:,adata.var.index=='LOC119965961'].layers['original_counts'].todense().flatten().tolist()[0]
adata.obs['strain_LOC119965947'] = adata[:,adata.var.index=='LOC119965947'].layers['original_counts'].todense().flatten().tolist()[0]
adata.obs['strain_LOC119970178'] = adata[:,adata.var.index=='LOC119970178'].layers['original_counts'].todense().flatten().tolist()[0]
adata.obs['strain_LOC119969773'] = adata[:,adata.var.index=='LOC119969773'].layers['original_counts'].todense().flatten().tolist()[0]

In [ ]:
for var in ['strain_LOC119965961','strain_LOC119965947','strain_LOC119970178','strain_LOC119969773']: #run for both G2M score and S score
    adata.var[var] = do_reg_parallel(adata.var.index, 
                                                        var,
                                                        10) #number of processes. 

In [ ]:
strain_gene_list = []
for var in ['strain_LOC119965961','strain_LOC119965947','strain_LOC119970178','strain_LOC119969773']:
    strain_gene_list = strain_gene_list + list(adata[:,adata.var[var]>0.05].var.index)

strain_gene_list = list(set(strain_gene_list))

with open('required_files/shark_strain_reg_genes.txt', 'w') as f:
    for line in strain_gene_list:
        f.write(f"{line}\n")

In [ ]:
adata.X = adata.layers['original_counts'].copy()
adata.raw = adata.copy()

In [ ]:
adata = adata[:,~adata.var.index.isin(strain_gene_list)].copy()

In [ ]:
adata.X = adata.layers['original_counts'].copy()
sc.pp.normalize_total(adata) # Normalizing to median total counts
sc.pp.log1p(adata) # Logarithmize the data
adata.layers["normalized_counts"] = adata.X.copy()

##highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=1000,)

##dimensionality reduction and clustering
sc.tl.pca(adata)
sc.pl.pca_variance_ratio(adata)

In [ ]:
sc.pp.neighbors(adata,n_pcs = 40)
sc.tl.umap(adata,negative_sample_rate = 3)
sc.tl.leiden(adata,resolution = 1, key_added = 'leiden_post_QC')
adata.write('h5ad_files/catshark/sScyCan1.1/SE8_9_pass_after_filtering_strain_corrected.h5ad')

In [ ]:
sc.pl.umap(adata,color = ['leiden_post_QC','sox10','LOC119970178','LOC119965947','LOC119965033','xbp1','LOC119961926','neurod1','tmie'],cmap = reds, vmin = 0.05,ncols = 3)